In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import time
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import wandb
from wandb.keras import WandbCallback

Using TensorFlow backend.


In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
data_dictionary = pd.read_csv("DataDictionaryWiDS2021.csv")
unlabeled = pd.read_csv("UnlabeledWiDS2021.csv")
training = pd.read_csv("TrainingWiDS2021.csv")

In [4]:
column_datatype_mapping = dict(zip(data_dictionary['Variable Name'], data_dictionary['Data Type']))

del training['Unnamed: 0']
del unlabeled['Unnamed: 0']

In [5]:
all_data = training.append(unlabeled).drop(['encounter_id', 
                                            'hospital_id', 
                                            'diabetes_mellitus'], axis=1)

all_data.shape

(140391, 177)

In [6]:
cat_cols = ['ethnicity', 'gender', 'hospital_admit_source',
           'icu_admit_source', 'icu_stay_type', 'icu_type',
           'apache_2_diagnosis', 'apache_3j_diagnosis','icu_id']


all_data = pd.get_dummies(all_data,dummy_na=True,columns=cat_cols)

'''
binary_cols = data_dictionary.loc[data_dictionary['Data Type'] == 'binary','Variable Name'].values
num_cols = data_dictionary.loc[data_dictionary['Data Type'] == 'numeric','Variable Name'].values 

for col in all_data.columns:
    if col in cat_cols:
        #print(col)
        all_data[col] = LabelEncoder().fit_transform(all_data[col].astype('str'))
        all_data[col]= all_data[col].astype('category')     '''
        
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(all_data.values)
all_data = pd.DataFrame(x_scaled)
all_data = all_data.fillna(0)

In [7]:
df_train = all_data[:len(training)]
df_pred = all_data[len(training):].reset_index(drop=True)
Y = training['diabetes_mellitus']

X_train, X_val, y_train, y_val = train_test_split(df_train, Y, test_size=0.20, random_state=42,shuffle=True )

In [8]:
earlystop_callback = EarlyStopping(
  monitor='val_accuracy', min_delta=0.0001,
  patience=10)

In [15]:
wandb.init(project="wids_2021", sync_tensorboard=True)
config = wandb.config
config.learning_rate = 0.0001
config.batch_size = 500
config.activation = 'relu'
config.optimizer = 'adam'
config.epochs = 29

tf.random.set_seed(87)

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1028,)),
    keras.layers.Dense(256, activation=tf.nn.relu,activity_regularizer=tf.keras.regularizers.L2(0.01)),
    tf.keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(128, activation=tf.nn.relu,activity_regularizer=tf.keras.regularizers.L2(0.01)),
    tf.keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(64, activation=tf.nn.relu,activity_regularizer=tf.keras.regularizers.L2(0.01)),
    tf.keras.layers.Dropout(rate=0.2),
    #keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

earlystop_callback = EarlyStopping(
  monitor='val_accuracy', min_delta=0.0001,
  patience=50)

'''lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=.0001,
    decay_steps=1000,
    decay_rate=0.9)
opt = keras.optimizers.Adam(learning_rate=lr_schedule)
'''
step = tf.Variable(0, trainable=False)
boundaries = [30, 60]
values = [0.0001, .00001, 0.000001]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)
learning_rate = learning_rate_fn(step)
opt = keras.optimizers.Nadam(learning_rate=learning_rate)

#opt = keras.optimizers.Adam(learning_rate=config.learning_rate)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(df_train, Y,
         epochs=config.epochs,
         batch_size=config.batch_size,
         validation_data=(X_val, y_val),
         #validation_freq=1,
         callbacks=[earlystop_callback, WandbCallback(validation_data=(X_val, y_val))])

#model.fit(X_train, y_train, 
#          epochs=config.epochs, 
#          batch_size=config.batch_size,
#          callbacks=[earlystop_callback, WandbCallback(validation_data=(X_val, y_val))])

metrics.roc_auc_score(y_val, model.predict(X_val))

epoch,49
loss,0.18155
accuracy,0.93413
val_loss,0.13171
val_accuracy,0.95974
_step,49
_runtime,2265
_timestamp,1614198014
best_val_loss,0.13138
best_epoch,48


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
accuracy,▁▂▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████
val_loss,█▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_accuracy,▁▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▇▇██
_timestamp,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▇▇██


wandb: wandb version 0.10.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/29
261/261 [==============================] - 12s 48ms/step - loss: 0.5718 - accuracy: 0.7753 - val_loss: 0.4959 - val_accuracy: 0.7832
Epoch 2/29
261/261 [==============================] - 12s 45ms/step - loss: 0.4762 - accuracy: 0.7955 - val_loss: 0.4473 - val_accuracy: 0.8185
Epoch 3/29
261/261 [==============================] - 12s 46ms/step - loss: 0.4449 - accuracy: 0.8198 - val_loss: 0.4264 - val_accuracy: 0.8280
Epoch 4/29
261/261 [==============================] - 11s 44ms/step - loss: 0.4301 - accuracy: 0.8265 - val_loss: 0.4149 - val_accuracy: 0.8322
Epoch 5/29
261/261 [==============================] - 10s 39ms/step - loss: 0.4210 - accuracy: 0.8291 - val_loss: 0.4052 - val_accuracy: 0.8364
Epoch 6/29
261/261 [==============================] - 11s 42ms/step - loss: 0.4112 - accuracy: 0.8333 - val_loss: 0.3962 - val_accuracy: 0.8395
Epoch 7/29
261/261 [==============================] - 12s 44ms/step - loss: 0.4040 - accuracy: 0.8353 - val_loss: 0.3885 - val_accuracy:

0.9650286957823331

In [10]:
#model.fit(X_train, y_train, epochs=50, batch_size=300)

In [11]:
metrics.roc_auc_score(y_val, model.predict(X_val))

0.9863086316807583

In [16]:
train_for_ensemble = pd.DataFrame(zip(Y.values,model.predict(df_train).reshape(-1)), columns=['diabetes_mellitus','pred'])
train_for_ensemble.to_csv('ensemble/TRAIN_Keras_29_epochs.csv')

submittion = pd.DataFrame([unlabeled.encounter_id,model.predict(df_pred).reshape(-1)]).T
submittion.encounter_id = submittion.encounter_id.astype('int32')
submittion.set_index('encounter_id',inplace=True)
submittion.columns = ['diabetes_mellitus']
submittion.to_csv('ensemble/SOLUTION_Keras_29_epochs.csv')

In [ ]:
submittion = pd.DataFrame([unlabeled.encounter_id,model.predict(df_pred).reshape(-1)]).T
submittion.encounter_id = submittion.encounter_id.astype('int32')
submittion.set_index('encounter_id',inplace=True)
submittion.columns = ['diabetes_mellitus']
submittion.fillna(0.5).to_csv('submissions/SolutionWiDS2021_Keras_29_epochs.csv')